In [119]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/09/18 14:37:09 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/18 14:37:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a38a5cc-0cbc-4618-a9d7-ff7b67a1a3db;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 169ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [3]:
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [4]:
objects=client.list_objects('raw', prefix='CAPES/', recursive=True)
csvs = []
for object in objects:
    csvs.append(object.object_name)

In [35]:
#FUNÇÃO PARA LISTAR AS COLUNAS IGUAIS DOS DOIS DATAFRAMES
def equalslist(df1, df2):
    return list(set(df1.columns).difference(set(df2.columns)))

#FUNÇÃO PARA JUNTAR DOIS DATAFRAMES, ESPECIFICANDO A KEY USADA E O MÉTODO DE JUNÇÃO 
def dfjoin(dataframe1, dataframe2, key, typejoin):
    
    equals = equalslist(dataframe1,dataframe2)
    equals.remove(key)

    #USANDO UM DATAFRAME AUXILIAR PARA NÃO MODIFICAR O DATAFRAME RAIZ CARREGADO
    dataframe3 = dataframe2
    for column in equals:
        dataframe3 = dataframe3.drop(column)
        
    dataframe_join = dataframe1.join(dataframe3, key, typejoin)

    #RETIRANDO AS LINHAS EM QUE A KEY É NULA
    dataframe_join = dataframe_join.dropna(subset=[key])
    return dataframe_join


In [6]:
#LISTANDO OS DATAFRAMES PRESENTES NO BUCKET RAW DO MinIO
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1
    

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv
1 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv
2 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv
3 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv
4 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv
5 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnic

In [117]:
def filterByColumn(name_database, database, client, column_key, key):
    for column_actual in database.columns:
        if column_actual == column_key:
            database = database.filter(F.col(column_key).like(key))
            database.toPandas().to_csv('temp.csv', index=False)
            client.fput_object('raw',f"UFMA/{name_database}", 'temp.csv')
            os.remove('temp.csv')
            break

In [118]:
bucket_name = 'raw'
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "SG_ENTIDADE_ENSINO", "%UFMA%")
    print(f"{csvs[csv_number]} OK")

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021, Bibliográfica Subtipo Artigo em Periódico.csv OK


CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2022, Bibliográfica Subtipo Artigo em Periódico.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2022, Bibliográfica Subtipo Trabalho em Anais.csv OK
CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil/Ano 2021.csv OK
CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil/Ano 2022.csv OK
CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil/Ano 2021.csv OK
CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil/Ano 2022.csv OK
CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao/Anos 2021 e 2022 - Subtipo Artigo em Periódico.csv OK


CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao/Anos 2021 e 2022 - Subtipo Livro.csv OK
CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao/Anos 2021 e 2022 - Subtipo Outro.csv OK
CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao/Anos 2021 e 2022 - Subtipo Trabalho em Anais.csv OK
CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu/Anos 2021 e 2022 - Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu/Anos 2021 e 2022 - Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu/Anos 2021 e 2022 - Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduaca

CAPES/2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano 2021.csv OK


CAPES/2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano 2022.csv OK
